<a href="https://colab.research.google.com/github/hussain0048/Research-Work/blob/main/Model_Training_Saad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1-Import library**

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import random
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
tf.__version__

'2.13.0'

#**2-Dataset Loading**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!',"'",]
print("Processing the intents..........")
path_to_save_model ="/content/gdrive/MyDrive/TrainedModel/"
with open('/content/gdrive/MyDrive/Proposal/Dataset/intentsnew (6).json') as json_data:
  intents = json.load(json_data)

Processing the intents..........


# **3-Data Preprocessing**

In [ ]:
print("Looping through intents to convert them to words, classes, documents and ignore_words.........")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        #add to our words list
        words.extend(w)
        # add to documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through intents to convert them to words, classes, documents and ignore_words.........


In [ ]:
# lemmaztize and lower each word and remove duplicates
nltk.download('wordnet')
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))


[nltk_data] Downloading package wordnet to /root/nltk_data...


305 documents
113 classes ['BCPLstand', 'C++', 'Campus changed', 'CandC++', 'Credit', 'Dev C++', 'IDE', 'Introduction C Programming', 'Introduction to  C/C+ Programming', 'LecCS201CS201P', 'OOP', 'Practical', 'Result', 'VSIDE', 'about', 'academic_support', 'addordropcourses', 'admission', 'admission_criteria', 'admission_process', 'admission_types', 'appearforexam', 'askquestionfromteacher', 'assignments', 'avoid_plagiarism', 'benefits', 'bestprogramlanguage', 'campuschange', 'career_services', 'cgpa', 'comparablecourses', 'complaint', 'concerncertificate', 'contact_professor', 'contactadmissiondepartment', 'continueeducationaftercompletingdefree', 'course selection', 'coursesyllabus', 'create account', 'datasavedisk', 'defragmentation', 'downloaddev', 'duplicateid', 'exam_system', 'exam_type', 'expensiveeducation', 'faculty_contact', 'fee_deadline', 'fee_installments', 'fee_structure', 'finalterm', 'financial_aid', 'fragmentation', 'freezeaccount', 'functions', 'functions type', 'func

In [ ]:
print("Creating the Data for Our Model......")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)
print("Creating Training Set, Bag of Words for our Model.....")

for document in documents:
  #initialize our bag of words
  bag = []
  #list of tokenized words for the pattern
  pattern_words = document[0]
  #lemmatize each word - create base word, in attempt to represent related words
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
  #create our bag of words array with 1, if word match found in current pattern
  for word in words:
    bag.append(1) if word in pattern_words else bag.append(0)

    #output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1

    training.append([bag, output_row])

#shuffle our features into np.array
random.shuffle(training)
training = np.array(training)

#create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

Creating the Data for Our Model......
Creating an List (Empty) for Output.....
Creating Training Set, Bag of Words for our Model.....
Training data created


# **4-Model Training**

## **4.1-Neural Netowrk**

### **4.1.1 Model Saving**

In [ ]:
model.save('chat_model_1', hist)
print("model created")

NameError: ignored

## **4.2- LSTM**

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Flatten,Dropout, BatchNormalization
from tensorflow.keras.layers import Bidirectional

### **4.2.1 - Model Architecture 1**

In [ ]:
#Create the LSTM network
model = Sequential()
model.add(LSTM(64, input_shape=(len(train_x[0]),1), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(16,return_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))
model.summary()
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
#adam = Adam(learning_rate=lr_schedule)
sgd = SGD(learning_rate=0.01 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=128, verbose=1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 391, 64)           16896     
                                                                 
 dropout_3 (Dropout)         (None, 391, 64)           0         
                                                                 
 batch_normalization_3 (Bat  (None, 391, 64)           256       
 chNormalization)                                                
                                                                 
 lstm_4 (LSTM)               (None, 391, 32)           12416     
                                                                 
 dropout_4 (Dropout)         (None, 391, 32)           0         
                                                                 
 batch_normalization_4 (Bat  (None, 391, 32)           128       
 chNormalization)                                     

### **4.2.2 - Model Architecture 2**

In [ ]:
#Create the LSTM network
model = Sequential()
model.add(LSTM(128, input_shape=(len(train_x[0]),1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(16,return_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))
model.summary()
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
lr_schedule = ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
#adam = Adam(learning_rate=lr_schedule)
sgd = SGD(learning_rate=0.001 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=128, verbose=1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 391, 128)          66560     
                                                                 
 dropout_4 (Dropout)         (None, 391, 128)          0         
                                                                 
 lstm_5 (LSTM)               (None, 391, 64)           49408     
                                                                 
 dropout_5 (Dropout)         (None, 391, 64)           0         
                                                                 
 batch_normalization_3 (Bat  (None, 391, 64)           256       
 chNormalization)                                                
                                                                 
 lstm_6 (LSTM)               (None, 391, 32)           12416     
                                                      

### 4.2.2.1 - Model Saving

In [ ]:
model.save( path_to_save_model+'chat_model_2', hist)
print("model created and saved")

model created and saved


### **4.2.3 - Model Architecture 3**

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(units=64,input_shape=(len(train_x[0]),1),return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=32,return_sequences=True)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=16,return_sequences=False)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))

# Build the model
model.build(input_shape=(None, len(train_x[0]), 1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 391, 128)          33792     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 391, 128)          0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 391, 64)           41216     
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 391, 64)           0         
                                                                 
 batch_normalization (Batch  (None, 391, 64)           256       
 Normalization)                                                  
                                                        

In [ ]:
sgd = SGD(learning_rate=0.001 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=128, verbose=1)

Epoch 1/100
932/932 [==============================] - 89s 80ms/step - loss: 4.5237 - accuracy: 0.0520
Epoch 2/100
932/932 [==============================] - 73s 79ms/step - loss: 3.7492 - accuracy: 0.1474
Epoch 3/100
932/932 [==============================] - 74s 79ms/step - loss: 3.2352 - accuracy: 0.2316
Epoch 4/100
932/932 [==============================] - 73s 79ms/step - loss: 2.7851 - accuracy: 0.3036
Epoch 5/100
932/932 [==============================] - 73s 79ms/step - loss: 2.4090 - accuracy: 0.3725
Epoch 6/100
932/932 [==============================] - 73s 78ms/step - loss: 2.0772 - accuracy: 0.4478
Epoch 7/100
932/932 [==============================] - 74s 79ms/step - loss: 1.7939 - accuracy: 0.5148
Epoch 8/100
932/932 [==============================] - 74s 80ms/step - loss: 1.5713 - accuracy: 0.5678
Epoch 9/100
932/932 [==============================] - 74s 80ms/step - loss: 1.3666 - accuracy: 0.6160
Epoch 10/100
932/932 [==============================] - 74s 80ms/step - l

In [ ]:
model.save( path_to_save_model+'chat_model_3', hist)
print("model created and saved")

model created and saved


### **4.2.4 - Model Architecture 4**

In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(units=128,input_shape=(len(train_x[0]),1),return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=64,return_sequences=True)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=32,return_sequences=True)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=16,return_sequences=False)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))

# Build the model
model.build(input_shape=(None, len(train_x[0]), 1))

model.summary()

sgd = SGD(learning_rate=0.001 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=128, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 391, 256)          133120    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 391, 256)          0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 391, 128)          164352    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 391, 128)          0         
                                                                 
 batch_normalization (Batch  (None, 391, 128)          512       
 Normalization)                                                  
                                                        

In [ ]:
model.save( path_to_save_model+'chat_model_4', hist)
print("model created and saved")

model created and saved


In [ ]:
model = Sequential()
model.add(Bidirectional(LSTM(units=128,input_shape=(len(train_x[0]),1),return_sequences=True)))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=64,return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(units=32,return_sequences=True)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(units=16,return_sequences=False)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(len(train_y[0]),activation="softmax"))

# Build the model
model.build(input_shape=(None, len(train_x[0]), 1))

model.summary()

sgd = SGD(learning_rate=0.001 ,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=128, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 391, 256)          133120    
 al)                                                             
                                                                 
 batch_normalization (Batch  (None, 391, 256)          1024      
 Normalization)                                                  
                                                                 
 bidirectional_1 (Bidirecti  (None, 391, 128)          164352    
 onal)                                                           
                                                                 
 dropout (Dropout)           (None, 391, 128)          0         
                                                                 
 bidirectional_2 (Bidirecti  (None, 391, 64)           41216     
 onal)                                                  

In [ ]:
model.save( path_to_save_model+'chat_model_5', hist)
print("model created and saved")